In [2]:
import pandas as pd 
import numpy as np
import os

In [3]:
df = pd.read_csv('./car+evaluation/car.csv')

In [4]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,class values
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   buying        1728 non-null   object
 1   maint         1728 non-null   object
 2   doors         1728 non-null   object
 3   persons       1728 non-null   object
 4   lug_boot      1728 non-null   object
 5   safety        1728 non-null   object
 6   class values  1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


---------

# **Transforming Doors Columns**

In [6]:
df['doors'].value_counts()

doors
2        432
3        432
4        432
5more    432
Name: count, dtype: int64

In [7]:
df['doors'] = df['doors'].str.split('m').str[0].astype(int)

In [8]:
df['doors'].value_counts()

doors
2    432
3    432
4    432
5    432
Name: count, dtype: int64

---------

## **Transforming Person Columns**

In [9]:
df['persons'].value_counts()

persons
2       576
4       576
more    576
Name: count, dtype: int64

In [10]:
df['persons'].mask(df['persons'] == 'more', '6', inplace=True)

In [11]:
df['persons'].value_counts()

persons
2    576
4    576
6    576
Name: count, dtype: int64

In [12]:
df['persons'] = df['persons'].astype(int)

------

# **Applying Ordinal Encoding**

In [13]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,class values
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [14]:
from sklearn.preprocessing import OrdinalEncoder

ordinal = OrdinalEncoder()

In [15]:
encode = ordinal.fit_transform(df[['buying','maint','lug_boot','safety','class values']])

In [16]:
encode

array([[3., 3., 2., 1., 2.],
       [3., 3., 2., 2., 2.],
       [3., 3., 2., 0., 2.],
       ...,
       [1., 1., 0., 1., 2.],
       [1., 1., 0., 2., 1.],
       [1., 1., 0., 0., 3.]])

In [17]:
encode_df =  pd.DataFrame(encode,columns=ordinal.get_feature_names_out())

In [18]:
encode_df

,buying,maint,lug_boot,safety,class values
0,3.0,3.0,2.0,1.0,2.0
1,3.0,3.0,2.0,2.0,2.0
2,3.0,3.0,2.0,0.0,2.0
3,3.0,3.0,1.0,1.0,2.0
4,3.0,3.0,1.0,2.0,2.0
...,...,...,...,...,...
1723,1.0,1.0,1.0,2.0,1.0
1724,1.0,1.0,1.0,0.0,3.0
1725,1.0,1.0,0.0,1.0,2.0
1726,1.0,1.0,0.0,2.0,1.0


In [19]:
df = df.drop(['buying','maint','lug_boot','safety','class values'],axis=1)

In [20]:
df = pd.concat([df,encode_df],axis=1)

In [21]:
df.head()

,doors,persons,buying,maint,lug_boot,safety,class values
0,2,2,3.0,3.0,2.0,1.0,2.0
1,2,2,3.0,3.0,2.0,2.0,2.0
2,2,2,3.0,3.0,2.0,0.0,2.0
3,2,2,3.0,3.0,1.0,1.0,2.0
4,2,2,3.0,3.0,1.0,2.0,2.0


In [22]:
df['buying'].value_counts()

buying
3.0    432
0.0    432
2.0    432
1.0    432
Name: count, dtype: int64

In [23]:
df['lug_boot'].value_counts()

lug_boot
2.0    576
1.0    576
0.0    576
Name: count, dtype: int64

In [24]:
df['safety'].value_counts()

safety
1.0    576
2.0    576
0.0    576
Name: count, dtype: int64

In [25]:
df['maint'].value_counts()

maint
3.0    432
0.0    432
2.0    432
1.0    432
Name: count, dtype: int64

In [26]:
df.to_csv('./car+evaluation/Car_Evaluation_final.csv',index=False)